In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1TdfOz-MVrA_XP8h8oXd8IBTvOXwbFN1Y',
                                    dest_path='/content/data.zip',
                                    unzip=True)

Unzipping...Done.


In [ ]:
import torchvision
import torch
from torch import nn
import torch.nn.functional as F
import torchvision.models as models
import torch.optim as optim
import copy
import os
from tqdm.autonotebook import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
import cv2
import sys

/usr/local/lib/python3.6/dist-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [ ]:
!git clone https://github.com/sanchit2843/Videoclassification
%cd Videoclassification

Cloning into 'Videoclassification'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 14 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (14/14), done.
/content/Videoclassification


In [ ]:
#Label file:
data_path = '/content/crime'
classes = os.listdir(data_path)
decoder = {}
for i in range(len(classes)):
    decoder[classes[i]] = i
encoder = {}
for i in range(len(classes)):
    encoder[i] = classes[i]

In [ ]:
id = list()
path = '/content/crime'
for i in os.listdir(path):
  p1 = os.path.join(path,i)
  for j in os.listdir(p1):
    p2 = os.path.join(p1,j)
    id.append((i,p2))

In [ ]:
class video_dataset(Dataset):
    def __init__(self,frame_list,sequence_length = 16,transform = None):
        self.frame_list = frame_list
        self.transform = transform
        self.sequence_length = sequence_length
    def __len__(self):
        return len(self.frame_list)
    def __getitem__(self,idx):
        label,path = self.frame_list[idx]
        img = cv2.imread(path)
        seq_img = list()
        for i in range(16):
          img1 = img[:,128*i:128*(i+1),:]
          if(self.transform):
            img1 = self.transform(img1)
          seq_img.append(img1)
        seq_image = torch.stack(seq_img)
        seq_image = seq_image.reshape(3,16,im_size,im_size)
        return seq_image,decoder[label]

In [ ]:
im_size = 128
mean = [0.4889, 0.4887, 0.4891]
std = [0.2074, 0.2074, 0.2074]


train_transforms = transforms.Compose([
                                        transforms.ToPILImage(),
                                        transforms.Resize((im_size,im_size)),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.RandomRotation(degrees=10),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean,std)])

train_data = video_dataset(id,sequence_length = 16,transform = train_transforms)
train_loader = DataLoader(train_data,batch_size = 8,num_workers = 4 ,shuffle = True)
dataloaders = {'train':train_loader}

In [ ]:
from model import resnet50
model = resnet50(class_num=8).to('cuda')

In [ ]:
from clr import *
device = 'cuda'
cls_criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum = 0.9,weight_decay = 1e-4)
num_epochs = 20
onecyc = OneCycle(len(train_loader)*num_epochs,1e-3)

In [ ]:
os.makedirs('/content/drive/My Drive/weights_crime',exist_ok = True)
from torch.autograd import Variable
iteration = 0
acc_all = list()
loss_all = list()

for epoch in range(num_epochs):
    print('')
    print(f"--- Epoch {epoch} ---")
    phase1 = dataloaders.keys()
    for phase in phase1:
        print('')
        print(f"--- Phase {phase} ---")
        epoch_metrics = {"loss": [], "acc": []}
        for batch_i, (X, y) in enumerate(dataloaders[phase]):
            #iteration = iteration+1
            image_sequences = Variable(X.to(device), requires_grad=True)
            labels = Variable(y.to(device), requires_grad=False)
            optimizer.zero_grad()
            #model.lstm.reset_hidden_state()
            predictions = model(image_sequences)
            loss = cls_criterion(predictions, labels)
            acc = 100 * (predictions.detach().argmax(1) == labels).cpu().numpy().mean()
            loss.backward()
            optimizer.step()
            epoch_metrics["loss"].append(loss.item())
            epoch_metrics["acc"].append(acc)
            if(phase=='train'):
                lr,mom = onecyc.calc()
                update_lr(optimizer, lr)
                update_mom(optimizer, mom)
            batches_done = epoch * len(dataloaders[phase]) + batch_i
            batches_left = num_epochs * len(dataloaders[phase]) - batches_done
            sys.stdout.write(
                    "\r[Epoch %d/%d] [Batch %d/%d] [Loss: %f (%f), Acc: %.2f%% (%.2f%%)]"
                    % (
                        epoch,
                        num_epochs,
                        batch_i,
                        len(dataloaders[phase]),
                        loss.item(),
                        np.mean(epoch_metrics["loss"]),
                        acc,
                        np.mean(epoch_metrics["acc"]),
                    )
                )

                # Empty cache
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

        print('')
        print('{} , acc: {}'.format(phase,np.mean(epoch_metrics["acc"])))
        torch.save(model.state_dict(),'/content/drive/My Drive/weights_crime/c3d_{}.h5'.format(epoch))
        if(phase=='train'):
          acc_all.append(np.mean(epoch_metrics["acc"]))
          loss_all.append(np.mean(epoch_metrics["loss"]))


--- Epoch 0 ---

--- Phase train ---
[Epoch 0/20] [Batch 437/438] [Loss: 2.001631 (1.511374), Acc: 0.00% (41.87%)]
train , acc: 41.86643835616438

--- Epoch 1 ---

--- Phase train ---
[Epoch 1/20] [Batch 437/438] [Loss: 1.151204 (1.305457), Acc: 50.00% (49.12%)]
train , acc: 49.11529680365297

--- Epoch 2 ---

--- Phase train ---
[Epoch 2/20] [Batch 437/438] [Loss: 1.231476 (1.179129), Acc: 50.00% (53.97%)]
train , acc: 53.96689497716895

--- Epoch 3 ---

--- Phase train ---
[Epoch 3/20] [Batch 437/438] [Loss: 0.774739 (1.041741), Acc: 75.00% (60.62%)]
train , acc: 60.61643835616438

--- Epoch 4 ---

--- Phase train ---
[Epoch 4/20] [Batch 437/438] [Loss: 0.276145 (0.809180), Acc: 75.00% (70.01%)]
train , acc: 70.00570776255708

--- Epoch 5 ---

--- Phase train ---
[Epoch 5/20] [Batch 437/438] [Loss: 1.330135 (0.614348), Acc: 50.00% (78.00%)]
train , acc: 77.99657534246575

--- Epoch 6 ---

--- Phase train ---
[Epoch 6/20] [Batch 437/438] [Loss: 1.023739 (0.401012), Acc: 75.00% (85.79

In [ ]:
def error_plot(loss):
    plt.figure(figsize=(10,5))
    plt.plot(loss)
    plt.title("Training loss plot")
    plt.xlabel("epochs")
    plt.ylabel("Loss")
    plt.show()
def acc_plot(acc):
    plt.figure(figsize=(10,5))
    plt.plot(acc)
    plt.title("Training accuracy plot")
    plt.xlabel("epochs")
    plt.ylabel("accuracy")
    plt.show()

In [ ]:
loss_all

[1.5113742563822499,
 1.305457015968349,
 1.17912877885174,
 1.041740705569585,
 0.8091797970201327,
 0.6143478534809531,
 0.4010123163461685,
 0.3424767658590726,
 0.22346783888666596,
 0.18811332580705756,
 0.1250801009538511,
 0.09738352649832425,
 0.07329675657291936,
 0.06421710863777491,
 0.03668630755927465,
 0.029382538033402673,
 0.02058421796587504,
 0.01861625916609481,
 0.012762131824340994,
 0.011054905242027213]

In [ ]:
error_plot(loss_all)

In [ ]:
acc_plot(acc_all)